In [1]:
import numpy
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD, GRO, XTC
from MDAnalysis.analysis.distances import distance_array
import MDAnalysisTests
import Bio
import Bio.PDB
import pytest
import pandas as pd
import biopandas
from biopandas.pdb import PandasPdb
import freesasa
import os.path
from MDAnalysis import AtomGroup
import subprocess
pd.options.display.max_columns = 999

In [2]:
KatG = mda.Universe("7ag8.pdb")
print(KatG)

<Universe with 10922 atoms>


C:\Users\user\anaconda3\lib\site-packages\MDAnalysis\coordinates\PDB.py:429: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


In [3]:
#AA_VOLUME (A^3) ('http://www.imgt.org/IMGTeducation/Aide-memoire/_UK/aminoacids/abbreviation.html')
#AA_MW (g/mol) ('https://www.thermofisher.com/uk/en/home/references/ambion-tech-support/rna-tools-and-calculators/proteins-and-amino-acids.html')
#AA_hydropathy index ('https://doi.org/10.1016/0022-2836(82)90515-0')
#AA_Pi ('https://www.sigmaaldrich.com/life-science/metabolomics/learning-center/amino-acid-reference-chart.html')

def aa_volume():
    aa_volumes = {'A': 88.6, 'R': 173.4, 'N': 114.1, 'D': 111.1, 'C': 108.5,
                 'Q': 143.8, 'E': 138.4, 'G': 60.1, 'H': 153.2, 'I': 166.7,
                 'L': 166.7, 'K': 168.6, 'M': 162.9, 'F': 189.9, 'P': 112.7,
                 'S': 89.0, 'T': 116.1, 'W': 227.8, 'Y': 193.6, 'V': 140.0}
    return aa_volumes


def MW():
    aa_MW = {'A': 89.1, 'R': 174.2, 'N': 132.1, 'D': 133.1, 'C': 121.1,
             'E': 147.1, 'Q': 146.2, 'G':75.1, 'H': 155.2, 'I': 131.2,
             'L': 131.2, 'K': 146.2, 'M': 149.2, 'F': 165.2, 'P': 115.1,
             'S': 105.1, 'T': 119.1, 'W': 204.2, 'Y': 181.2, 'V': 117.1}
    return aa_MW


def hydropathy():
    aa_hydropathy_index = {'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5,
                           'C': 2.5, 'E': -3.5, 'Q': -3.5, 'G': -0.4,
                           'H': -3.2, 'I': 4.5, 'L': 3.8, 'K': -3.9,
                           'M': 1.9, 'F': 2.8, 'P': -1.6, 'S': -0.8, 
                           'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2}
    return aa_hydropathy_index


def Pi():
    
    aa_Pi = {'A': 6, 'R': 10.76, 'N': 5.41, 'D': 2.77, 'C': 5.07, 'E': 3.22,
             'Q': 5.65, 'G': 5.97, 'H': 7.59, 'I': 6.02, 'L': 5.98, 'K': 9.74,
             'M': 5.74, 'F': 5.48, 'P': 6.3, 'S': 5.68, 'T': 5.6, 'W': 5.89,
             'Y': 5.66, 'V': 5.96}
    return aa_Pi

In [4]:
def d_chem_features():
    '''calculates change in biochemical feature for every possible mutation'''
    
    #loads in aa attributes
    volume_dict = aa_volume()
    MW_dict = MW()
    hydropathy_dict = hydropathy()
    Pi_dict = Pi()
    
    #creates df from attribute function data
    df = pd.DataFrame.from_dict([volume_dict, MW_dict, hydropathy_dict, Pi_dict]).T.reset_index(0)
    df.rename(columns = {'index':'Amino_acid', 0:'volume', 1:'MW', 2:'hydropathy',
                        3:'Pi'}, inplace=True)
    
    #creates dicts for delta attributes from df
    d_volume_dict, d_Pi_dict, d_MW_dict, d_hydropathy_dict = {}, {}, {}, {}
    for i in df.index:
        for j in df.index:
            mutation = df['Amino_acid'][i] + df['Amino_acid'][j]
            d_volume_dict[mutation] = df['volume'][i] - df['volume'][j]
            d_MW_dict[mutation] = df['MW'][i] - df['MW'][j]
            d_hydropathy_dict[mutation] = df['hydropathy'][i] - df['hydropathy'][j]
            d_Pi_dict[mutation] = df['Pi'][i] - df['Pi'][j]        
    
    #creates df from delta dictionaries
    d_chem_df = pd.DataFrame.from_dict([d_volume_dict, d_MW_dict, d_hydropathy_dict, d_Pi_dict]).T.reset_index(0)
    d_chem_df.rename(columns = {'index':'dAA', 0:'d_volume', 1:'d_MW', 2:'d_hydropathy',
                        3:'d_Pi'}, inplace=True)
    return d_chem_df

In [5]:
d_chem_features()

,dAA,d_volume,d_MW,d_hydropathy,d_Pi
0,AA,0.0,0.0,0.0,0.00
1,AR,-84.8,-85.1,6.3,-4.76
2,AN,-25.5,-43.0,5.3,0.59
3,AD,-22.5,-44.0,5.3,3.23
4,AC,-19.9,-32.0,-0.7,0.93
...,...,...,...,...,...
395,VS,51.0,12.0,5.0,0.28
396,VT,23.9,-2.0,4.9,0.36
397,VW,-87.8,-87.1,5.1,0.07
398,VY,-53.6,-64.1,5.5,0.30


In [6]:
#extracts KatG seq
from Bio import SeqIO
for record in SeqIO.parse("rcsb_pdb_7AG8.fasta","fasta"):
    print(record.id)
records = SeqIO.parse("rcsb_pdb_7AG8.fasta","fasta")

7AG8_1|Chains


In [7]:
KatG_seq = str(record.seq)
aa_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

In [8]:
#loop to find identify all KatG mutations 
resnum = 0
KatG_mutations = []

for i in KatG_seq:
    resnum = resnum + 1
    for aa in aa_list:
        if aa == i:
            print('')
        else:
            ia, resnuma, aaa = str(i), str(resnum), str(aa)
            df_add = (ia+resnuma+aaa)
            KatG_mutations.append(df_add)

In [9]:
#dataframe with all possible KatG chain A mutations 
data = {'KatG mutations': KatG_mutations}
KatG_mutations_df = pd.DataFrame (data, columns = ['KatG mutations'])

In [10]:
#identifies the centre of mass of each residue in chain A
resid_no = 0
residue_no_A = []
residue_CoM_A = []
chain_A = []

for i in KatG_seq:
    resid_no = resid_no + 1
    residue_no_A.append(i + str(resid_no))
    resid_no_string = str(resid_no)
    select_atoms_string = 'resid '+ resid_no_string + ' and segid A'
    atom_coordinates = KatG.select_atoms(select_atoms_string)
    residue_CoM_A.append(atom_coordinates.center_of_mass())
    chain_A.append('A')
    
data = {'Residue': residue_no_A, 'Chain': chain_A, 'Centre of mass': residue_CoM_A}
residue_CoM_A_df = pd.DataFrame (data, columns = ['Residue','Chain','Centre of mass'])
print(residue_CoM_A_df)


    Residue Chain                                     Centre of mass
0        M1     A                                                 []
1        P2     A                                                 []
2        E3     A                                                 []
3        Q4     A                                                 []
4        H5     A                                                 []
..      ...   ...                                                ...
735    R736     A  [101.71184776487075, 134.8153974993157, 136.44...
736    F737     A  [104.94849311732212, 129.8637529291616, 138.16...
737    D738     A  [99.68625343362409, 132.32714759696879, 139.98...
738    V739     A  [98.00821963702562, 131.03244203077924, 136.44...
739    R740     A  [101.54049856367341, 128.31209317416, 134.9765...

[740 rows x 3 columns]


In [11]:
#identifies the centre of mass of each residue in chain B
resid_no = 0
residue_no_B = []
residue_CoM_B = []
chain_B = []

for i in KatG_seq:
    resid_no = resid_no + 1
    residue_no_B.append(i + str(resid_no))
    resid_no_string = str(resid_no)
    select_atoms_string = 'resid '+ resid_no_string + ' and segid B'
    atom_coordinates = KatG.select_atoms(select_atoms_string)
    residue_CoM_B.append(atom_coordinates.center_of_mass())
    chain_B.append('B')
    
data = {'Residue': residue_no_B, 'Chain': chain_B, 'Centre of mass': residue_CoM_B}
residue_CoM_B_df = pd.DataFrame (data, columns = ['Residue','Chain','Centre of mass'])
print(residue_CoM_B_df)

    Residue Chain                                     Centre of mass
0        M1     B                                                 []
1        P2     B                                                 []
2        E3     B                                                 []
3        Q4     B                                                 []
4        H5     B                                                 []
..      ...   ...                                                ...
735    R736     B  [102.56951717105538, 106.68485590620568, 102.0...
736    F737     B  [106.1688799400712, 111.39674823454799, 100.59...
737    D738     B  [100.97355567927137, 109.2970553620259, 98.230...
738    V739     B  [99.15080305467927, 110.36249033674973, 101.78...
739    R740     B  [102.86387110467739, 112.88535360473507, 103.8...

[740 rows x 3 columns]


In [12]:
# adds the chain A and chain B dataframes together
residue_CoM_df = residue_CoM_A_df.append(residue_CoM_B_df, ignore_index = True)

In [13]:
residue_CoM_df

,Residue,Chain,Centre of mass
0,M1,A,[]
1,P2,A,[]
2,E3,A,[]
3,Q4,A,[]
4,H5,A,[]
...,...,...,...
1475,R736,B,"[102.56951717105538, 106.68485590620568, 102.0..."
1476,F737,B,"[106.1688799400712, 111.39674823454799, 100.59..."
1477,D738,B,"[100.97355567927137, 109.2970553620259, 98.230..."
1478,V739,B,"[99.15080305467927, 110.36249033674973, 101.78..."


In [14]:
#Identifies distance from expected INH binding sites (Site 1) for the residues in each chain

#Find Ser315 centre of mass in protomer A and B
Ser315_A = KatG.select_atoms('resid 315 and segid A')
Ser315_B = KatG.select_atoms('resid 315 and segid B')
Ser315_A_CoM = Ser315_A.center_of_mass()
Ser315_B_CoM = Ser315_B.center_of_mass()
print('Centre of mass for Ser315 on protomer A:', Ser315_A_CoM)
print('Centre of mass for Ser315 on protomer B:', Ser315_B_CoM)

#Find Asp137 centre of mass in protomer A and B
Asp137_A = KatG.select_atoms('resid 137 and segid A')
Asp137_B = KatG.select_atoms('resid 137 and segid B')
Asp137_A_CoM = Asp137_A.center_of_mass()
Asp137_B_CoM = Asp137_B.center_of_mass()
print('Centre of mass for Asp137 on protomer A:', Asp137_A_CoM)
print('Centre of mass for Asp137 on protomer B:', Asp137_B_CoM)

#Identify Hem atoms in protomer A and B
Hem_A = KatG.select_atoms('resid 801 and segid A')
Hem_B = KatG.select_atoms('resid 801 and segid B')

#'Site1_A' is an initial centre of mass in protomer A used to identify the Hem atoms which are closest
#to the suspected Isoniazid binding site
Site1_A = KatG.select_atoms('resid 315 and segid A', 'resid 137 and segid A')
Site1_A_CoM = Site1_A.center_of_mass()
haem_distances_A = []
haem_atom_number_A = []

#Identifies distance of each Hem atoms from suspected binding site
for i in Hem_A:
    atom_position = i.position
    dist = numpy.linalg.norm(Site1_A_CoM-atom_position)
    haem_distances_A.append(dist)
    haem_atom_number_A.append(str(i)[6:11])
    
#Creates dataframe to show atom number and distance
data = {'Atom number': haem_atom_number_A, 'Distance from Site 1 A': haem_distances_A}
distance_site1_A_df = pd.DataFrame (data, columns = ['Atom number', 'Distance from Site 1 A'])
distance_site1_A_df = distance_site1_A_df.set_index('Atom number')
print(distance_site1_A_df)

#Uses a carbon from each of Hem, Ser315 and Asp137, close to the suspected binding site
#to find a new, more accurate, binding site location
Site1_A_true = KatG.select_atoms('bynum 10844','bynum 2112','bynum 890')
Site1_A_true_CoM = Site1_A_true.center_of_mass()
print('\n','New site A:''\n', Site1_A_true_CoM)

#'Site1_B' is an initial centre of mass in protomer B used to identify the Hem atoms which are closest
#to the suspected Isoniazid binding site
Site1_B = KatG.select_atoms('resid 315 and segid B', 'resid 137 and segid B')
Site1_B_CoM = Site1_B.center_of_mass()
haem_distances_B = []
haem_atom_number_B = []

#Identifies distance of each Hem atoms from suspected binding site
for i in Hem_B:
    atom_position = i.position
    dist = numpy.linalg.norm(Site1_B_CoM-atom_position)
    haem_distances_B.append(dist)
    haem_atom_number_B.append(str(i)[6:11])
    
#Creates dataframe to show atom number and distance
data = {'Atom number': haem_atom_number_B, 'Distance from Site 1 B': haem_distances_B}
distance_site1_B_df = pd.DataFrame (data, columns = ['Atom number', 'Distance from Site 1 B'])
distance_site1_B_df = distance_site1_B_df.set_index('Atom number')
print(distance_site1_B_df)

#Uses a carbon from each of Hem, Ser315 and Asp137, close to the suspected binding site
#to find a new, more accurate, binding site location
Site1_B_true = KatG.select_atoms('bynum 10911','bynum 7582','bynum 6236')
Site1_B_true_CoM = Site1_B_true.center_of_mass()
print('\n','New site B:''\n', Site1_B_true_CoM)

Centre of mass for Ser315 on protomer A: [137.71549016 110.062874   149.22819452]
Centre of mass for Ser315 on protomer B: [140.31562649 129.74415378  92.06069301]
Centre of mass for Asp137 on protomer A: [133.16871676 115.19637174 144.94393794]
Centre of mass for Asp137 on protomer B: [135.49328217 125.00875238  96.04659678]
             Distance from Site 1 A
Atom number                        
10837                      8.124812
10838                      6.996807
10839                     10.965349
10840                     11.538067
10841                      7.232062
10842                      6.167269
10843                      5.783919
10844                      6.675063
10845                      4.980348
10846                      5.944127
10847                      5.700443
10848                      5.917667
10849                      6.626282
10850                      5.762992
10851                      8.072774
10852                      8.848015
10853                   

In [15]:
#adds a new column for distance from site 1 for each residue in that given chain
#Note - Site 1 chain B is done later as only residues in chain A will be used for ML model
Site1_coord = []
for i in residue_CoM_df['Chain']:
    if (i == 'A'):
        Site1_coord.append(Site1_A_true_CoM)
    else:
        Site1_coord.append(Site1_B_true_CoM)
residue_CoM_df['Site 1 Coordinates'] = Site1_coord

Site1_dist = []
Site1_dist_no = -1

for i in residue_CoM_df['Centre of mass']:
    Site1_dist_no = Site1_dist_no + 1 
    chain = residue_CoM_df.loc[Site1_dist_no,'Chain']
    CoM = residue_CoM_df.loc[Site1_dist_no,'Centre of mass']
    if len(CoM) == 0:
        Site1_dist.append('No centre of mass')
    else:
        if chain == 'A':
            dist = numpy.linalg.norm(Site1_A_CoM-i)
            Site1_dist.append(dist)
        else:
            dist = numpy.linalg.norm(Site1_B_CoM-i)
            Site1_dist.append(dist)
            
residue_CoM_df['Distance from Site 1'] = Site1_dist
residue_CoM_df 

,Residue,Chain,Centre of mass,Site 1 Coordinates,Distance from Site 1
0,M1,A,[],"[135.5568728593392, 113.46202070912435, 148.68...",No centre of mass
1,P2,A,[],"[135.5568728593392, 113.46202070912435, 148.68...",No centre of mass
2,E3,A,[],"[135.5568728593392, 113.46202070912435, 148.68...",No centre of mass
3,Q4,A,[],"[135.5568728593392, 113.46202070912435, 148.68...",No centre of mass
4,H5,A,[],"[135.5568728593392, 113.46202070912435, 148.68...",No centre of mass
...,...,...,...,...,...
1475,R736,B,"[102.56951717105538, 106.68485590620568, 102.0...","[137.7765238725133, 126.92174901271466, 92.186...",41.195753
1476,F737,B,"[106.1688799400712, 111.39674823454799, 100.59...","[137.7765238725133, 126.92174901271466, 92.186...",35.615301
1477,D738,B,"[100.97355567927137, 109.2970553620259, 98.230...","[137.7765238725133, 126.92174901271466, 92.186...",40.836819
1478,V739,B,"[99.15080305467927, 110.36249033674973, 101.78...","[137.7765238725133, 126.92174901271466, 92.186...",42.519218


In [16]:
#creates a dataframe for all mutations in chain A
KatG_mutations_chain_A = []
for i in KatG_mutations_df['KatG mutations']:
    KatG_mutations_chain_A.append('A')

KatG_mutations_df['Chain'] =  KatG_mutations_chain_A

In [17]:
KatG_mutations_df_B = KatG_mutations_df.copy()

In [18]:
#creates a dataframe for all mutations in chain B
del KatG_mutations_df_B['Chain']
KatG_mutations_chain_B = []
for i in KatG_mutations_df_B['KatG mutations']:
    KatG_mutations_chain_B.append('B')

KatG_mutations_df_B['Chain'] =  KatG_mutations_chain_B

In [19]:
#joins chain A and chain B dataframes together
KatG_mutations_df = KatG_mutations_df.append(KatG_mutations_df_B, ignore_index = True)

In [20]:
#creates unique ID for each possible KatG mutation

Unique_ID = []
mutation_no = -1

for i in KatG_mutations_df['KatG mutations']:
    mutation_no = mutation_no + 1
    if mutation_no < 14060:
        ID = i + ' ' + 'A'
    else:
        ID = i + ' ' + 'B'
    Unique_ID.append(ID)

KatG_mutations_df['Unique ID'] =  Unique_ID
KatG_mutations_df


,KatG mutations,Chain,Unique ID
0,M1A,A,M1A A
1,M1C,A,M1C A
2,M1D,A,M1D A
3,M1E,A,M1E A
4,M1F,A,M1F A
...,...,...,...
28115,R740S,B,R740S B
28116,R740T,B,R740T B
28117,R740V,B,R740V B
28118,R740W,B,R740W B


In [21]:
#adds distance from site 1 column

KatG_mutations_dist = []
mutation_no = -1

#cycles through all possible KatG mutations
for a in KatG_mutations_df['Unique ID']:
    mutation_no = mutation_no + 1
    #cycles through all KatG residues looking for a match to the residue being mutated
    for b in residue_CoM_df['Residue']:
        if a[:-3] == b:
            if mutation_no < 14060:
                row_no = residue_CoM_df[residue_CoM_df['Residue']==b].index[0]
            else:
                row_no = residue_CoM_df[residue_CoM_df['Residue']==b].index[1]
            Site1_dist = residue_CoM_df.loc[row_no,'Distance from Site 1']
    KatG_mutations_dist.append(Site1_dist)
    
KatG_mutations_df['Distance from Site 1'] =  KatG_mutations_dist
KatG_mutations_df

,KatG mutations,Chain,Unique ID,Distance from Site 1
0,M1A,A,M1A A,No centre of mass
1,M1C,A,M1C A,No centre of mass
2,M1D,A,M1D A,No centre of mass
3,M1E,A,M1E A,No centre of mass
4,M1F,A,M1F A,No centre of mass
...,...,...,...,...
28115,R740S,B,R740S B,38.661813
28116,R740T,B,R740T B,38.661813
28117,R740V,B,R740V B,38.661813
28118,R740W,B,R740W B,38.661813


In [22]:
d_chem_features = d_chem_features()

In [23]:
#add d_chem_features columns to KatG_mutations_df
d_volume = []
d_MW = []
d_hydropathy = []
d_Pi = []

for a in KatG_mutations_df['KatG mutations']:
    res1 = a[0]
    res2 = a[-1]
    AA = res1 + res2
    for b in d_chem_features['dAA']:
        if str(AA) == str(b):
            row_no = d_chem_features[d_chem_features['dAA']==b].index[0]
            vol = d_chem_features.loc[row_no,'d_volume']
            d_volume.append(vol)
            MW = d_chem_features.loc[row_no,'d_MW']
            d_MW.append(MW)
            hydrop =d_chem_features.loc[row_no,'d_hydropathy']
            d_hydropathy.append(hydrop)
            pi =d_chem_features.loc[row_no,'d_Pi']
            d_Pi.append(pi)
            
KatG_mutations_df['d_volume'] = d_volume
KatG_mutations_df['d_MW'] = d_MW
KatG_mutations_df['d_hydropathy'] = d_hydropathy
KatG_mutations_df['d_Pi'] = d_Pi

KatG_mutations_df

,KatG mutations,Chain,Unique ID,Distance from Site 1,d_volume,d_MW,d_hydropathy,d_Pi
0,M1A,A,M1A A,No centre of mass,74.3,60.1,0.1,-0.26
1,M1C,A,M1C A,No centre of mass,54.4,28.1,-0.6,0.67
2,M1D,A,M1D A,No centre of mass,51.8,16.1,5.4,2.97
3,M1E,A,M1E A,No centre of mass,24.5,2.1,5.4,2.52
4,M1F,A,M1F A,No centre of mass,-27.0,-16.0,-0.9,0.26
...,...,...,...,...,...,...,...,...
28115,R740S,B,R740S B,38.661813,84.4,69.1,-3.7,5.08
28116,R740T,B,R740T B,38.661813,57.3,55.1,-3.8,5.16
28117,R740V,B,R740V B,38.661813,33.4,57.1,-8.7,4.80
28118,R740W,B,R740W B,38.661813,-54.4,-30.0,-3.6,4.87


In [24]:
#creates a list of all possible codons
codon_list = []
bases = ['T','A','C','G']
for i in bases:
    for h in bases:
        for z in bases:
            codon = (i,h,z)
            codon_list.append(codon)

In [25]:
#function to convert lists to strings
def ListToString(s):
    str1 = ''
    for ele in s:
        str1 += ele
    return str1 

In [26]:
#creates dataframe with all possible codon to codon mutations
codon_list_start = []
codon_list_end = []
for i in codon_list:
    for a in codon_list:
        start_codon = ListToString(i)
        end_codon = ListToString(a)
        codon_list_start.append(start_codon)
        codon_list_end.append(end_codon)
        
data = {'Start codon': codon_list_start, 'End codon': codon_list_end}
codon_df = pd.DataFrame (data, columns = ['Start codon', 'End codon'])
codon_df
    

,Start codon,End codon
0,TTT,TTT
1,TTT,TTA
2,TTT,TTC
3,TTT,TTG
4,TTT,TAT
...,...,...
4091,GGG,GCG
4092,GGG,GGT
4093,GGG,GGA
4094,GGG,GGC


In [27]:
#checks if code can determine the number of bases changed in a codon mutation
codon_1 = 'TAC'
codon_2 = 'GCC'
base_no = -1
errors = 0
for i in codon_1:
    base_no = base_no + 1
    if codon_2[int(base_no)] == i:
        errors += 0
    else:
        errors += 1
        
print(errors)
        

2


In [28]:
#adds column for number of substitutions for each codon mutation
substitutions = []
row_no = -1
for i in codon_list_start:
    codon_1 = i
    row_no = row_no + 1
    codon_2 = codon_df.loc[row_no,'End codon']
    base_no = -1
    subs = 0
    for a in codon_1:
        base_no = base_no + 1
        if codon_2[int(base_no)] == a:
            subs += 0
        else:
            subs += 1
    substitutions.append(subs)

codon_df['Number of substitutions'] = substitutions 
codon_df    

,Start codon,End codon,Number of substitutions
0,TTT,TTT,0
1,TTT,TTA,1
2,TTT,TTC,1
3,TTT,TTG,1
4,TTT,TAT,1
...,...,...,...
4091,GGG,GCG,1
4092,GGG,GGT,1
4093,GGG,GGA,1
4094,GGG,GGC,1


In [29]:
#loads dataframe showing what residue each codon codes for - but has spaces and additional rows (draft)
aa_codons_drft = pd.read_csv(r'C:\Users\user\anaconda3\envs\mdaenv\Inh_resistance\amino_acid_codons.csv')

In [30]:
#converts draft codon df to formatted df
aa_codons = aa_codons_drft.iloc[:-3,]
aa_codons = aa_codons.rename(columns ={0:'Codon',1:'Amino Acid'})
aa_codons.set_index('Codon ', inplace = True)
aa_codons

,Amino Acid
Codon,
TTT,F
TTA,L
TTC,F
TTG,L
TAT,Y
...,...
GCG,A
GGT,G
GGA,G


In [31]:
#adds start and end residues for each mutation to main df
start_aa = []
end_aa = []
for i in KatG_mutations_df['KatG mutations']:
    start = i[0]
    end = i[-1]
    start_aa.append(start)
    end_aa.append(end)
KatG_mutations_df['Start residue'] = start_aa
KatG_mutations_df['End residue'] = end_aa
KatG_mutations_df

,KatG mutations,Chain,Unique ID,Distance from Site 1,d_volume,d_MW,d_hydropathy,d_Pi,Start residue,End residue
0,M1A,A,M1A A,No centre of mass,74.3,60.1,0.1,-0.26,M,A
1,M1C,A,M1C A,No centre of mass,54.4,28.1,-0.6,0.67,M,C
2,M1D,A,M1D A,No centre of mass,51.8,16.1,5.4,2.97,M,D
3,M1E,A,M1E A,No centre of mass,24.5,2.1,5.4,2.52,M,E
4,M1F,A,M1F A,No centre of mass,-27.0,-16.0,-0.9,0.26,M,F
...,...,...,...,...,...,...,...,...,...,...
28115,R740S,B,R740S B,38.661813,84.4,69.1,-3.7,5.08,R,S
28116,R740T,B,R740T B,38.661813,57.3,55.1,-3.8,5.16,R,T
28117,R740V,B,R740V B,38.661813,33.4,57.1,-8.7,4.80,R,V
28118,R740W,B,R740W B,38.661813,-54.4,-30.0,-3.6,4.87,R,W


In [32]:
#sets index before joining 2 dataframes
codon_df.set_index('Start codon', inplace = True)

In [33]:
#creates dataframe which shows the starting amino acid for each codon mutation
combined_codon_df = codon_df.join(aa_codons)
combined_codon_df.reset_index(inplace = True)
combined_codon_df.rename(columns = {'Amino Acid':'Start AA','index':'Start codon'}, inplace = True)
combined_codon_df

,Start codon,End codon,Number of substitutions,Start AA
0,AAA,TTT,3,K
1,AAA,TTA,2,K
2,AAA,TTC,3,K
3,AAA,TTG,3,K
4,AAA,TAT,2,K
...,...,...,...,...
4091,TTT,GCG,3,F
4092,TTT,GGT,2,F
4093,TTT,GGA,3,F
4094,TTT,GGC,3,F


In [34]:
#adds end amino acid onto the dataframe
combined_codon_df.set_index('End codon', inplace = True)
combined_codon_df = combined_codon_df.join(aa_codons)
combined_codon_df.reset_index(inplace = True)
combined_codon_df.rename(columns = {'Amino Acid':'End AA','index':'End codon'}, inplace = True)
combined_codon_df

,End codon,Start codon,Number of substitutions,Start AA,End AA
0,AAA,AAA,0,K,K
1,AAA,AAC,1,N,K
2,AAA,AAG,1,K,K
3,AAA,AAT,1,N,K
4,AAA,ACA,1,T,K
...,...,...,...,...,...
4091,TTT,TGT,1,C,F
4092,TTT,TTA,1,L,F
4093,TTT,TTC,1,F,F
4094,TTT,TTG,1,L,F


In [35]:
#breaks down the number of substitutions based on the residue mutation
combined_codon_df[['Start AA', 'End AA', 'Number of substitutions']].groupby(['Start AA','End AA']).agg(['min','max',numpy.mean,'count'])

Number of substitutions                    
                                    min max      mean count
Start AA End AA                                            
A        A                            0   1  0.750000    16
         C                            2   3  2.750000     8
         D                            1   2  1.750000     8
         E                            1   2  1.750000     8
         F                            2   3  2.750000     8
...                                 ...  ..       ...   ...
Y        Stop                         1   2  1.333333     6
         T                            2   3  2.750000     8
         V                            2   3  2.750000     8
         W                            2   2  2.000000     2
         Y                            0   1  0.500000     4

[441 rows x 4 columns]

In [36]:
#adds start and end amino acid together in 1 column to get the mutation 
mutation_1 = []
row_no_1 = -1
for s_aa in combined_codon_df['Start AA']:
    row_no_1 = row_no_1 + 1 
    e_aa = combined_codon_df.loc[row_no_1,'End AA']
    d_aa = str(s_aa + e_aa)
    mutation_1.append(d_aa)
combined_codon_df['Mutation'] = mutation_1
combined_codon_df.set_index('Mutation', inplace = True)
combined_codon_df
    

,End codon,Start codon,Number of substitutions,Start AA,End AA
Mutation,,,,,
KK,AAA,AAA,0,K,K
NK,AAA,AAC,1,N,K
KK,AAA,AAG,1,K,K
NK,AAA,AAT,1,N,K
TK,AAA,ACA,1,T,K
...,...,...,...,...,...
CF,TTT,TGT,1,C,F
LF,TTT,TTA,1,L,F
FF,TTT,TTC,1,F,F


In [37]:
#adds start and end amino acid together to get the mutation
KatG_mutations_df
mutation_2 = []
row_no_2 = -1 
for s_aa in KatG_mutations_df['Start residue']:
    row_no_2 = row_no_2 + 1
    e_aa = KatG_mutations_df.loc[row_no_2,'End residue']
    d_aa = str(s_aa + e_aa)
    mutation_2.append(d_aa)
KatG_mutations_df['Mutation'] = mutation_2
KatG_mutations_df.set_index('Mutation', inplace = True)
KatG_mutations_df

,KatG mutations,Chain,Unique ID,Distance from Site 1,d_volume,d_MW,d_hydropathy,d_Pi,Start residue,End residue
Mutation,,,,,,,,,,
MA,M1A,A,M1A A,No centre of mass,74.3,60.1,0.1,-0.26,M,A
MC,M1C,A,M1C A,No centre of mass,54.4,28.1,-0.6,0.67,M,C
MD,M1D,A,M1D A,No centre of mass,51.8,16.1,5.4,2.97,M,D
ME,M1E,A,M1E A,No centre of mass,24.5,2.1,5.4,2.52,M,E
MF,M1F,A,M1F A,No centre of mass,-27.0,-16.0,-0.9,0.26,M,F
...,...,...,...,...,...,...,...,...,...,...
RS,R740S,B,R740S B,38.661813,84.4,69.1,-3.7,5.08,R,S
RT,R740T,B,R740T B,38.661813,57.3,55.1,-3.8,5.16,R,T
RV,R740V,B,R740V B,38.661813,33.4,57.1,-8.7,4.80,R,V


In [38]:
#creates df to show how the number of substitutions varies for each mutation
combined_codon_df.reset_index(inplace = True)
subs_df = combined_codon_df[['Mutation', 'Number of substitutions']].groupby(['Mutation']).agg(['min','max','count'])
subs_df.reset_index(inplace = True)
subs_df.set_index('Mutation', inplace = True)
subs_df


Number of substitutions          
                             min max count
Mutation                                  
AA                             0   1    16
AC                             2   3     8
AD                             1   2     8
AE                             1   2     8
AF                             2   3     8
...                          ...  ..   ...
YStop                          1   2     6
YT                             2   3     8
YV                             2   3     8
YW                             2   2     2
YY                             0   1     4

[441 rows x 3 columns]

In [39]:
#adds number of substitutions for each possible mutation in the main df
KatG_mutations_df = KatG_mutations_df.join(subs_df)

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


In [40]:
KatG_mutations_df

,KatG mutations,Chain,Unique ID,Distance from Site 1,d_volume,d_MW,d_hydropathy,d_Pi,Start residue,End residue,"(Number of substitutions, min)","(Number of substitutions, max)","(Number of substitutions, count)"
Mutation,,,,,,,,,,,,,
AC,A15C,A,A15C A,No centre of mass,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
AC,A16C,A,A16C A,No centre of mass,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
AC,A53C,A,A53C A,35.218287,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
AC,A55C,A,A55C A,31.791704,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
AC,A60C,A,A60C A,38.926044,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW,Y597W,B,Y597W B,40.613433,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2
YW,Y608W,B,Y608W B,33.528173,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2
YW,Y638W,B,Y638W B,46.007273,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2


In [41]:
#resets main index on main df
KatG_mutations_df.reset_index(inplace=True)
KatG_mutations_df

,Mutation,KatG mutations,Chain,Unique ID,Distance from Site 1,d_volume,d_MW,d_hydropathy,d_Pi,Start residue,End residue,"(Number of substitutions, min)","(Number of substitutions, max)","(Number of substitutions, count)"
0,AC,A15C,A,A15C A,No centre of mass,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
1,AC,A16C,A,A16C A,No centre of mass,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
2,AC,A53C,A,A53C A,35.218287,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
3,AC,A55C,A,A55C A,31.791704,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
4,AC,A60C,A,A60C A,38.926044,-19.9,-32.0,-0.7,0.93,A,C,2,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28115,YW,Y597W,B,Y597W B,40.613433,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2
28116,YW,Y608W,B,Y608W B,33.528173,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2
28117,YW,Y638W,B,Y638W B,46.007273,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2
28118,YW,Y678W,B,Y678W B,48.498844,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2


In [42]:
#freesasa code

In [43]:
#calculates SASA for each atom in KatG using edited PDB with only CA altLoc B removed
KatG_edit = mda.Universe("7ag8_R418_edit.pdb")
structure = freesasa.Structure("7ag8_R418_edit.pdb")
result = freesasa.calc(structure)

ASA_dict_atom = {}
for i in range(0, structure.nAtoms()):
    ASA_dict_atom[i] = result.atomArea(i)
    
SASA_df = pd.DataFrame.from_dict(ASA_dict_atom, orient='index', columns = ['atom_sasa'])

C:\Users\user\anaconda3\lib\site-packages\MDAnalysis\coordinates\PDB.py:429: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


In [44]:
#dict for each residues code
res_codes = {'ALA':'A', 'ARG':'R', 'ASN':'N', 'ASP':'D', 'CYS':'C', 
             'GLU':'E', 'GLN':'Q', 'GLY':'G', 'HIS':'H', 'ILE':'I', 
             'LEU':'L', 'LYS':'K', 'MET':'M', 'PHE':'F', 'PRO':'P', 
             'SER':'S', 'THR':'T', 'TRP':'W', 'TYR':'Y', 'VAL':'V'}

#define max total sasa for each residue (also useful to define aa)
maxasa_dict = {'ALA':129, 'ARG':274, 'ASN':195, 'ASP':193, 'CYS':167, 'GLU':223,
                'GLN':225, 'GLY':104, 'HIS':224, 'ILE':197, 'LEU':201, 'LYS':236,
                'MET':224, 'PHE':240, 'PRO':159, 'SER':155, 'THR':172, 'TRP':285,
                'TYR':263, 'VAL':174}

In [45]:
    #creates df with each atoms SASA and a unique combined ID 
    residue_list, atom_list, residue_number, chain_label, combined_ID = [], [], [], [], []
    for i in range(0, structure.nAtoms()):
        residue_list.append(structure.residueName(i))
        atom_list.append(structure.atomName(i))
        residue_number.append(structure.residueNumber(i))
        chain_label.append(structure.chainLabel(i))
        j = 0
        if int(structure.residueNumber(i)) < 10:
            j = '000' + str(structure.residueNumber(i)).strip()
        elif int(structure.residueNumber(i)) < 100:
            j = '00' + str(structure.residueNumber(i)).strip()
        elif int(structure.residueNumber(i)) < 1000:
            j = '0' + str(structure.residueNumber(i)).strip()
        else:
            j = structure.residueNumber(i).strip()
        combined_ID.append(structure.chainLabel(i) + '_' + str(j) + '_' + structure.residueName(i))

    SASA_df['atom'] = atom_list
    SASA_df['residue'] = residue_list
    SASA_df['residue_number'] = residue_number
    SASA_df['chain'] = chain_label
    SASA_df['combined_ID'] = combined_ID
    
    SASA_df

,atom_sasa,atom,residue,residue_number,chain,combined_ID
0,26.778193,N,GLY,24,A,A_0024_GLY
1,41.526774,CA,GLY,24,A,A_0024_GLY
2,1.935937,C,GLY,24,A,A_0024_GLY
3,30.605271,O,GLY,24,A,A_0024_GLY
4,0.000000,N,HIS,25,A,A_0025_HIS
...,...,...,...,...,...,...
10815,6.324353,NE,ARG,740,B,B_0740_ARG
10816,1.726302,CZ,ARG,740,B,B_0740_ARG
10817,2.320039,NH1,ARG,740,B,B_0740_ARG
10818,20.751627,NH2,ARG,740,B,B_0740_ARG


In [46]:
    #creates dataframe with the sum of the SASA of all atoms in each residue
    residue_list, residue_ID_list, CA_atom_ID_list = [], [], []
    Dict = {}
    SASA_sum = 0

    for i in SASA_df.index:
        try:
            if SASA_df['combined_ID'][i] == SASA_df['combined_ID'][i+1]:
                #atoms in same residue, so add current sasa and keep iterating
                SASA_sum += SASA_df['atom_sasa'][i]
            else:
                #atoms not in same residue, so add sasa and reset SASA_sum
                SASA_sum += SASA_df['atom_sasa'][i]
                Dict[SASA_df['combined_ID'][i]] = SASA_sum
                SASA_sum = 0
                residue_list.append(SASA_df['residue'][i])
                residue_ID_list.append(SASA_df['residue_number'][i])
        except KeyError:
            #for last index of df, so add sasa and reset SASA_sum
            SASA_sum += SASA_df['atom_sasa'][i]
            Dict[SASA_df['combined_ID'][i]] = SASA_sum
            SASA_sum = 0
            residue_list.append(SASA_df['residue'][i])
            residue_ID_list.append(SASA_df['residue_number'][i])
        if SASA_df['atom'][i].strip() == 'CA':
            #list indexes of CA 
            CA_atom_ID_list.append(SASA_df.index[i])
    
    sasa_sum_df = pd.DataFrame.from_dict(Dict, orient='index', 
                columns=['SASA_residue']).reset_index(0).rename(columns={'index':'combined_ID'})
    sasa_sum_df['residue_ID'] = residue_ID_list
    sasa_sum_df['CA_ID'] = CA_atom_ID_list
    
    #insert predefined max sasa for each residue
    maxasa_list = []
    for i in sasa_sum_df['combined_ID']:
        for key, value in maxasa_dict.items():
            if i[7:] == key:
                maxasa_list.append(value)
    sasa_sum_df['maxASA'] = maxasa_list
    
    sasa_sum_df

,combined_ID,SASA_residue,residue_ID,CA_ID,maxASA
0,A_0024_GLY,100.846174,24,1,104
1,A_0025_HIS,127.637993,25,5,224
2,A_0026_MET,18.262181,26,15,224
3,A_0027_LYS,61.530393,27,23,236
4,A_0028_TYR,39.727088,28,32,263
...,...,...,...,...,...
1412,B_0736_ARG,0.125640,736,10772,274
1413,B_0737_PHE,34.730453,737,10783,240
1414,B_0738_ASP,79.101138,738,10794,193
1415,B_0739_VAL,81.799088,739,10802,174


In [47]:
    #calculate and insert relative SASA for each residue
    sasa_sum_df['RSA'] = sasa_sum_df['SASA_residue']/sasa_sum_df['maxASA']
    #filter for residues with SASA > 0.25 as surface residues -- based on KatG surface area residues and RNAP calc
    surface_residues_df = sasa_sum_df[sasa_sum_df['RSA'] >= 0.26].reset_index(0)
    surface_residues_df

,index,combined_ID,SASA_residue,residue_ID,CA_ID,maxASA,RSA
0,0,A_0024_GLY,100.846174,24,1,104,0.969675
1,1,A_0025_HIS,127.637993,25,5,224,0.569812
2,3,A_0027_LYS,61.530393,27,23,236,0.260722
3,13,A_0037_ASP,53.610179,37,96,193,0.277773
4,16,A_0040_PRO,77.247684,40,132,159,0.485834
...,...,...,...,...,...,...,...
418,1402,B_0726_ALA,63.733677,726,10692,129,0.494060
419,1406,B_0730_LYS,65.592781,730,10724,236,0.277936
420,1414,B_0738_ASP,79.101138,738,10794,193,0.409850
421,1415,B_0739_VAL,81.799088,739,10802,174,0.470110


In [48]:
    #identifies how close each non surface alpha carbon is from a surface alpha carbon, denoted as depth
    CA = KatG_edit.select_atoms('name CA')
    sa_CA = AtomGroup(surface_residues_df['CA_ID'], KatG_edit)
    distances = distance_array(CA.positions, sa_CA.positions)
    min_distances = []
    for i in distances:
        min_distances.append(min(i))
distance_df = pd.DataFrame(min_distances, columns=['Depth'])

In [49]:
    #insert chain and numbering
    chain_list, pdb_residue_list, resname_list = [], [], []
    for i in CA:
        chain_list.append(str(i.segment)[9])
        pdb_residue_list.append(i.resid)
        resname_list.append(i.resname)
    #create and fill depth dataframe
    distance_df['pdb_residue'] = pdb_residue_list
    distance_df['pdb_chain'] = chain_list

In [50]:
    #converts resnames to rescode IDs (ALA --> A e.g)
    rescode_list = []
    for i in resname_list:
        for k, v in res_codes.items():
            if i == k:
                rescode_list.append(v)
    distance_df['rescode'] = rescode_list

In [51]:
#adds unique ID to dpeth dataframe
distance_df_unique_ID = []
distance_df_row_no = -1
for i in distance_df['Depth']:
    distance_df_row_no += 1
    Res = distance_df.loc[distance_df_row_no,'rescode']
    Num = distance_df.loc[distance_df_row_no,'pdb_residue']
    Chain = distance_df.loc[distance_df_row_no,'pdb_chain']
    Unique = str(Res+str(Num)+'_'+Chain)
    distance_df_unique_ID.append(Unique)
distance_df['Residue Unique ID'] = distance_df_unique_ID

In [52]:
KatG_mutations_df.rename(columns={'Unique ID':'Mutation Unique ID'}, inplace = True)

In [53]:
#adds comparable unique ID to main df
Residue_unique_ID = []
for i in KatG_mutations_df['Mutation Unique ID']:
    Res = str(i)[:-3]
    Chain = str(i)[-1]
    Unique = str(Res+'_'+Chain)
    Residue_unique_ID.append(Unique)
    
KatG_mutations_df['Residue Unique ID'] = Residue_unique_ID

In [54]:
#removes characteristic columns in depth df so only unique ID and depth are left
distance_merge = distance_df.copy()
del distance_merge['pdb_residue'],distance_merge['pdb_chain'],distance_merge['rescode']

In [55]:
KatG_mutations_df.reset_index(inplace = True)
del KatG_mutations_df['index']
KatG_mutations_df

,Mutation,KatG mutations,Chain,Mutation Unique ID,Distance from Site 1,d_volume,d_MW,d_hydropathy,d_Pi,Start residue,End residue,"(Number of substitutions, min)","(Number of substitutions, max)","(Number of substitutions, count)",Residue Unique ID
0,AC,A15C,A,A15C A,No centre of mass,-19.9,-32.0,-0.7,0.93,A,C,2,3,8,A15_A
1,AC,A16C,A,A16C A,No centre of mass,-19.9,-32.0,-0.7,0.93,A,C,2,3,8,A16_A
2,AC,A53C,A,A53C A,35.218287,-19.9,-32.0,-0.7,0.93,A,C,2,3,8,A53_A
3,AC,A55C,A,A55C A,31.791704,-19.9,-32.0,-0.7,0.93,A,C,2,3,8,A55_A
4,AC,A60C,A,A60C A,38.926044,-19.9,-32.0,-0.7,0.93,A,C,2,3,8,A60_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28115,YW,Y597W,B,Y597W B,40.613433,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2,Y597_B
28116,YW,Y608W,B,Y608W B,33.528173,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2,Y608_B
28117,YW,Y638W,B,Y638W B,46.007273,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2,Y638_B
28118,YW,Y678W,B,Y678W B,48.498844,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2,Y678_B


In [56]:
#adds depth column df to main df
KatG_mutations_df.set_index('Residue Unique ID', inplace = True)
distance_merge.set_index('Residue Unique ID', inplace = True)
KatG_mutations_df = KatG_mutations_df.join(distance_merge)
KatG_mutations_df.reset_index(inplace = True)

In [57]:
#loads in DSSP csv file for KatG secondary structure 
KatG_ss = pd.read_csv(r'C:\Users\user\anaconda3\envs\mdaenv\Inh_resistance\KatG_DSSP_ss.csv')
KatG_ss = KatG_ss.iloc[:,:4]

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,25

In [58]:
#adds unique ID to ss dataframe
ss_df_unique_ID = []
ss_df_row_no = -1
for i in KatG_ss['Res_number']:
    ss_df_row_no += 1
    Res = KatG_ss.loc[ss_df_row_no,'Res']
    Num = KatG_ss.loc[ss_df_row_no,'Res_number']
    Chain = KatG_ss.loc[ss_df_row_no,'Chain']
    Unique = str(Res+str(Num)+'_'+Chain)
    ss_df_unique_ID.append(Unique)
KatG_ss['Residue_Unique_ID'] = ss_df_unique_ID

In [59]:
# gives each DSSP category a new column for a binary readout if the residue fits into that ss
List = ['H','O','T','S','G','E','B','I']
Dict = {}

for i in List:
    Dict[i] = []
    
for i in KatG_ss['SS']:
    for j in List:
        if i == j:
            Dict[j].append('1')
        else:
            Dict[j].append('0')
            
for i in List:
    KatG_ss[i] = Dict[i]

In [60]:
# deletes columns to ensure no overlap with main characteristic df 
KatG_ss.drop(['Chain','Res_number','Res'], inplace = True, axis = 1)

In [61]:
KatG_ss.set_index('Residue_Unique_ID', inplace = True)
KatG_mutations_df.set_index('Residue Unique ID', inplace = True)
KatG_mutations_df = KatG_mutations_df.join(KatG_ss)
KatG_mutations_df.reset_index(inplace = True)
KatG_mutations_df.rename(columns = {'index':'Residue Unique ID'}, inplace = True)

In [62]:
unique_id = []
tf = []

for i in range(1,741):
    Res = KatG_seq[i-1]
    Num = i 
    #append residue info in chain A
    Unique = str(Res+str(Num)+'_A')
    unique_id.append(Unique)
    res_tf = KatG.select_atoms('resid '+ str(i) +' and segid A').tempfactors.mean()
    tf.append(res_tf)
    #append residue info in chain B
    Unique = str(Res+str(Num)+'_B')
    unique_id.append(Unique)
    res_tf = KatG.select_atoms('resid '+ str(i) +' and segid B').tempfactors.mean()
    tf.append(res_tf)
    
data = {'Residue_Unique_ID':unique_id, 'Tempfactor':tf}
tf_df = pd.DataFrame (data, columns = ['Residue_Unique_ID','Tempfactor'])

C:\Users\user\AppData\Local\Temp/ipykernel_19516/3423963753.py:10: RuntimeWarning: Mean of empty slice.
  res_tf = KatG.select_atoms('resid '+ str(i) +' and segid A').tempfactors.mean()
C:\Users\user\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\user\AppData\Local\Temp/ipykernel_19516/3423963753.py:15: RuntimeWarning: Mean of empty slice.
  res_tf = KatG.select_atoms('resid '+ str(i) +' and segid B').tempfactors.mean()


In [63]:
tf_df.set_index('Residue_Unique_ID', inplace = True)
KatG_mutations_df.set_index('Residue Unique ID', inplace = True)
KatG_mutations_df = KatG_mutations_df.join(tf_df)
KatG_mutations_df.reset_index(inplace = True)
KatG_mutations_df.rename(columns = {'index':'Residue Unique ID'}, inplace = True)

In [64]:
# distance from heam 

In [65]:
Site1_A = KatG.select_atoms('resid 315 and segid A', 'resid 137 and segid A')
Site1_A_CoM = Site1_A.center_of_mass()
haem_distances_A = []

In [66]:
KatG.select_atoms('resid 315 and segid A').center_of_mass()

array([137.71549016, 110.062874  , 149.22819452])

In [67]:
unique_id = []
Heam_dist = []
Hem_A_CoM = Hem_A.center_of_mass()
Hem_B_CoM = Hem_B.center_of_mass()

for i in range(1,741):
    Res = KatG_seq[i-1]
    Num = i 
    #append residue info in chain A
    Unique = str(Res+str(Num)+'_A')
    unique_id.append(Unique)
    res_CoM = KatG.select_atoms('resid '+ str(i) +' and segid A').center_of_mass()
    dist = numpy.linalg.norm(res_CoM-Hem_A_CoM)
    Heam_dist.append(dist)
    #append residue info in chain B
    Unique = str(Res+str(Num)+'_B')
    unique_id.append(Unique)
    res_CoM = KatG.select_atoms('resid '+ str(i) +' and segid B').center_of_mass()
    dist = numpy.linalg.norm(res_CoM-Hem_B_CoM)    
    Heam_dist.append(dist)
    
data = {'Residue_Unique_ID':unique_id, 'Hem_dist':Heam_dist}
Heam_dist_df = pd.DataFrame (data, columns = ['Residue_Unique_ID','Hem_dist'])

In [68]:
Heam_dist_df.set_index('Residue_Unique_ID', inplace = True)
KatG_mutations_df.set_index('Residue Unique ID', inplace = True)
KatG_mutations_df = KatG_mutations_df.join(Heam_dist_df)
KatG_mutations_df.reset_index(inplace = True)
KatG_mutations_df.rename(columns = {'index':'Residue Unique ID'}, inplace = True)

In [69]:
#Site2_Distance - only in chain A as chain A is only used for features in ML model 

In [70]:
unique_id = []
Site2_A_dist = []
Site2_A_CoM = [130, 105, 151]
# derived from KatG structure paper

for i in range(1,741):
    Res = KatG_seq[i-1]
    Num = i 
    #append residue info in chain A
    Unique = str(Res+str(Num)+'_A')
    unique_id.append(Unique)
    res_CoM = KatG.select_atoms('resid '+ str(i) +' and segid A').center_of_mass()
    dist = numpy.linalg.norm(res_CoM-Site2_A_CoM)
    Site2_A_dist.append(dist)
    #append residue info in chain B
    Unique = str(Res+str(Num)+'_B')
    unique_id.append(Unique)
    res_CoM = KatG.select_atoms('resid '+ str(i) +' and segid B').center_of_mass()
    dist = numpy.linalg.norm(res_CoM-Site2_A_CoM)    
    Site2_A_dist.append(dist)
    
data = {'Residue_Unique_ID':unique_id, 'Site2_dist':Site2_A_dist}
Site2_dist_df = pd.DataFrame (data, columns = ['Residue_Unique_ID','Site2_dist'])

In [71]:
Site2_dist_df.set_index('Residue_Unique_ID', inplace = True)
KatG_mutations_df.set_index('Residue Unique ID', inplace = True)
KatG_mutations_df = KatG_mutations_df.join(Site2_dist_df)
KatG_mutations_df.reset_index(inplace = True)
KatG_mutations_df.rename(columns = {'index':'Residue Unique ID'}, inplace = True)

In [72]:
# use SNAP2 to determine link between genotype and phentoype of given mutations

In [73]:
SNAP2_pd = pd.read_csv('SNAP2_values.csv')
SNAP2_pd.drop(['Predicted Effect','Expected Accuracy'], inplace = True, axis = 1)
SNAP2_pd.set_index('Variant', inplace = True)
SNAP2_pd.rename(columns = {'Score':'SNAP2_score'}, inplace = True)

In [74]:
SNAP2_pd

,SNAP2_score
Variant,
M1A,-83
M1R,-85
M1N,-84
M1D,-65
M1C,-93
...,...
R740S,14
R740T,3
R740W,77


In [75]:
KatG_mutations_df.set_index('KatG mutations', inplace = True)
KatG_mutations_df = KatG_mutations_df.join(SNAP2_pd)
KatG_mutations_df.reset_index(inplace = True)
KatG_mutations_df.rename(columns = {'index':'KatG mutations'}, inplace = True)

In [76]:
KatG_mutations_df

,KatG mutations,Residue Unique ID,Mutation,Chain,Mutation Unique ID,Distance from Site 1,d_volume,d_MW,d_hydropathy,d_Pi,Start residue,End residue,"(Number of substitutions, min)","(Number of substitutions, max)","(Number of substitutions, count)",Depth,SS,H,O,T,S,G,E,B,I,Tempfactor,Hem_dist,Site2_dist,SNAP2_score
0,A106C,A106_A,AC,A,A106C A,14.629585,-19.9,-32.0,-0.7,0.93,A,C,2,3,8,13.382314,H,1,0,0,0,0,0,0,0,82.300003,10.589587,14.275680,-27
1,A106C,A106_B,AC,B,A106C B,14.465213,-19.9,-32.0,-0.7,0.93,A,C,2,3,8,10.619754,H,1,0,0,0,0,0,0,0,91.110001,10.373444,62.981121,-27
2,A106D,A106_A,AD,A,A106D A,14.629585,-22.5,-44.0,5.3,3.23,A,D,1,2,8,13.382314,H,1,0,0,0,0,0,0,0,82.300003,10.589587,14.275680,5
3,A106D,A106_B,AD,B,A106D B,14.465213,-22.5,-44.0,5.3,3.23,A,D,1,2,8,10.619754,H,1,0,0,0,0,0,0,0,91.110001,10.373444,62.981121,5
4,A106E,A106_A,AE,A,A106E A,14.629585,-49.8,-58.0,5.3,2.78,A,E,1,2,8,13.382314,H,1,0,0,0,0,0,0,0,82.300003,10.589587,14.275680,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28115,Y98T,Y98_B,YT,B,Y98T B,17.788426,77.5,62.1,-0.6,0.06,Y,T,2,3,8,4.926100,O,0,1,0,0,0,0,0,0,93.639999,15.165136,71.318767,41
28116,Y98V,Y98_A,YV,A,Y98V A,17.955051,53.6,64.1,-5.5,-0.30,Y,V,2,3,8,6.643434,O,0,1,0,0,0,0,0,0,88.239998,15.225818,8.741972,34
28117,Y98V,Y98_B,YV,B,Y98V B,17.788426,53.6,64.1,-5.5,-0.30,Y,V,2,3,8,4.926100,O,0,1,0,0,0,0,0,0,93.639999,15.165136,71.318767,34
28118,Y98W,Y98_A,YW,A,Y98W A,17.955051,-34.2,-23.0,-0.4,-0.23,Y,W,2,2,2,6.643434,O,0,1,0,0,0,0,0,0,88.239998,15.225818,8.741972,22


In [77]:
KatG_mutations_df.to_csv('KatG_mutations_df.csv')

In [128]:
#distance between site 1 and site 2
Site1 =  [135.55687286, 113.46202071, 148.68011475]
Site2 = [130, 105, 151]
x = numpy.asarray(Site1)
y = numpy.asarray(Site2)
dist = numpy.linalg.norm(x-y)
print(dist)

10.385879743759846
